# Movie Recommender System

## 1. Introduction
This notebook builds a movie recommender system using the MovieLens 1M dataset. We will use a collaborative filtering approach based on matrix factorization to predict user ratings for movies and generate recommendations.

## 2. Data Loading and Preparation

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# Load the MovieLens 1M dataset
ratings_data, info = tfds.load('movielens/1m-ratings', with_info=True)
ratings = tfds.as_dataframe(ratings_data['train'])

# Extract user IDs, movie titles, and ratings
user_ids = ratings['user_id'].unique()
movie_titles = ratings['movie_title'].unique()

# Create user and movie mappers
user_to_index = {user: i for i, user in enumerate(user_ids)}
movie_to_index = {movie: i for i, movie in enumerate(movie_titles)}

ratings['user_index'] = ratings['user_id'].map(user_to_index)
ratings['movie_index'] = ratings['movie_title'].map(movie_to_index)

ratings.head()

## 3. Model Building

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.models import Model

embedding_dim = 50
num_users = len(user_ids)
num_movies = len(movie_titles)

# User input
user_input = Input(shape=(1,))
user_embedding = Embedding(num_users, embedding_dim, input_length=1)(user_input)
user_vec = Flatten()(user_embedding)

# Movie input
movie_input = Input(shape=(1,))
movie_embedding = Embedding(num_movies, embedding_dim, input_length=1)(movie_input)
movie_vec = Flatten()(movie_embedding)

# Dot product of user and movie embeddings
dot_product = Dot(axes=1)([user_vec, movie_vec])

# Model
model = Model([user_input, movie_input], dot_product)
model.summary()

## 4. Model Compilation and Training

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

# Prepare data for training
X = ratings[['user_index', 'movie_index']].values
y = ratings['user_rating'].values

history = model.fit([X[:, 0], X[:, 1]], y, epochs=10, batch_size=64, validation_split=0.1)

## 5. Model Evaluation and Recommendation Generation

In [ ]:
# Function to recommend movies for a user
def recommend_movies(user_id, num_recommendations=10):
    user_idx = user_to_index[user_id]
    movie_indices = np.array(list(movie_to_index.values()))
    user_idx_array = np.array([user_idx for _ in range(len(movie_indices))])
    
    predictions = model.predict([user_idx_array, movie_indices])
    
    # Get top N recommendations
    top_indices = predictions.flatten().argsort()[-num_recommendations:][::-1]
    
    # Get movie titles from indices
    recommended_movies = [list(movie_to_index.keys())[i] for i in top_indices]
    
    return recommended_movies

# Example: Get recommendations for user 1
recommendations = recommend_movies(b'1')
print(f'Recommendations for user 1: {recommendations}')

## 6. Conclusion
This notebook demonstrates how to build a movie recommender system using collaborative filtering with matrix factorization. The model learns embeddings for users and movies to predict ratings and generate personalized recommendations. This approach can be extended with more complex models and features to further improve recommendation quality.